# Finance Use Case: Loan Approval Explanation

**Goal:** Explain why a specific loan applicant was rejected (or predicted to default) using SHAP.


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
import matplotlib.pyplot as plt

# Initialize JS
shap.initjs()

In [ ]:
# 1. Generate Synthetic Data
np.random.seed(42)
n_samples = 2000

data = pd.DataFrame({
    "Income_Annual": np.random.normal(50000, 15000, n_samples).astype(int),
    "Credit_Score": np.random.normal(650, 100, n_samples).astype(int),
    "Debt_to_Income": np.random.uniform(0.1, 0.6, n_samples),
    "Loan_Amount": np.random.normal(15000, 5000, n_samples).astype(int),
    "Years_Employed": np.random.randint(0, 20, n_samples),
    "Has_Prior_Default": np.random.choice([0, 1], n_samples, p=[0.9, 0.1])
})

risk_score = (
    (data["Debt_to_Income"] * 100) - 
    (data["Credit_Score"] / 10) - 
    (data["Income_Annual"] / 2000) + 
    (data["Has_Prior_Default"] * 50)
)
risk_score += np.random.normal(0, 10, n_samples)
data["Defaulted"] = (risk_score > -20).astype(int)
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(data.drop("Defaulted", axis=1), data["Defaulted"])
print("Model Trained.")

In [ ]:
# 2. Compute Shapley Values
explainer = shap.TreeExplainer(model)
shap_values = explainer(data.drop("Defaulted", axis=1))

In [ ]:
# 3. Global Interpretability
plt.figure(figsize=(10, 6))
shap.plots.beeswarm(shap_values, show=True)

In [ ]:
# 4. Local Interpretability (Waterfall)
high_risk_idx = np.where(data["Defaulted"] == 1)[0][0]
plt.figure(figsize=(10, 6))
shap.plots.waterfall(shap_values[high_risk_idx], show=True)